Projeto em PySpark para dados de preços de combustíveis da ANP.

O trabalho faz uso de dados publicados pela ANP (Agência Nacional de Petróleo)

> Em cumprimento às determinações da Lei do Petróleo (Lei nº 9478/1997, artigo 8º), a ANP acompanha os preços praticados por revendedores de combustíveis automotivos e de gás liquefeito de petróleo envasilhado em botijões de 13 quilos (GLP P13), por meio de uma pesquisa semanal de preços realizada por empresa contratada.

- [Série Histórica de Preços de Combustíveis e de GLP](https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/serie-historica-de-precos-de-combustiveis)

Coletamos a série histórica de "Combustíveis automotivos" que vai de 2004 a 2023. São 39 arquivos CSV totalizando aproximadamente 3.7 GB.

![Amostra dos dados](./assets/amostra_planilha_revenda.png)


Aproveitando a configuração de cluster da Google Cloud Platform criada para o [projeto Hadoop](https://github.com/pinei/hadoop-precos-combustiveis), incluímos o JupyterLab para trabalhar com o PySpark.

![cluster-info](./assets/cluster_info.png)

### Ingestão

Os arquivos CSV já estão organizados em um bucket do Google Cloud Storage.

> `https://console.cloud.google.com/storage/browser/hadoop-dados-brutos`

Optamos por usar uma API Python para trazer os arquivos do storage e trabalhar com eles no cluster ou na máquina local (a depender de como estamos executando este notebook).

In [1]:
!pip install google-cloud-storage

  Obtaining dependency information for google-cloud-storage from https://files.pythonhosted.org/packages/68/b8/cb00819641313e67b640857ffff2c5afdcdcfb4940def9f1502bf614a6d9/google_cloud_storage-2.12.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-auth<3.0dev,>=2.23.3 from https://files.pythonhosted.org/packages/39/7c/2e4fa55a99f83ef9ef229ac5d59c44ceb90e2d0145711590c0fa39669f32/google_auth-2.23.3-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-api-core!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.0,<3.0.0dev,>=1.31.5 from https://files.pythonhosted.org/packages/4d/ce/4fd62ea66b3508debc795e475336ce915929765870f0ad52328426ba016e/google_api_core-2.12.0-py3-none-any.whl.metadata
  Obtaining dependency information for google-cloud-core<3.0dev,>=2.3.0 from https://files.pythonhosted.org/packages/a2/40/02045f776fdb6e44194f34b6375a26ce8a61bd9bd03cd8930ed91cf51a62/google_cloud_core-2.3.3-py2.py3-none-any.whl.metadata
  Obtaining dependency information for

A API precisa de uma chave para autenticação no serviço.

Passo a passo para gerar a chave:
- Console do Google Cloud Platform
- APIs e Serviços > Credenciais
- Criar Conta de Serviço
- Criar Chave do tipo JSON

> "É feito o download de um arquivo contendo a chave privada. Armazene o arquivo com segurança porque essa chave não pode ser recuperada em caso de perda."

Subimos o arquivo JSON para uma pasta de trabalho no cluster:

> `/home/aldinei_bastos/work/seventh-abacus-395221-52fc140a5609.json`

In [7]:
import os

home_dir = os.environ["HOME"]
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = f'{home_dir}/work/seventh-abacus-395221-52fc140a5609.json'

In [8]:
from google.cloud import storage

# Inicializa o cliente
storage_client = storage.Client()

# Lista todos os buckets
buckets = list(storage_client.list_buckets())

# Obtem um bucket específico
bucket = storage_client.get_bucket('hadoop-dados-brutos')
blobs = list(bucket.list_blobs())

display(blobs)

[<Blob: hadoop-dados-brutos, anp-combustiveis-automotivos/, 1693781005419814>,
 <Blob: hadoop-dados-brutos, anp-combustiveis-automotivos/ca-2004-01.csv, 1693781239357851>,
 <Blob: hadoop-dados-brutos, anp-combustiveis-automotivos/ca-2004-02.csv, 1693781397853813>,
 <Blob: hadoop-dados-brutos, anp-combustiveis-automotivos/ca-2005-01.csv, 1693781399574637>,
 <Blob: hadoop-dados-brutos, anp-combustiveis-automotivos/ca-2005-02.csv, 1693781397708461>,
 <Blob: hadoop-dados-brutos, anp-combustiveis-automotivos/ca-2006-01.csv, 1693781398243286>,
 <Blob: hadoop-dados-brutos, anp-combustiveis-automotivos/ca-2006-02.csv, 1693781398142808>,
 <Blob: hadoop-dados-brutos, anp-combustiveis-automotivos/ca-2007-01.csv, 1693781395022902>,
 <Blob: hadoop-dados-brutos, anp-combustiveis-automotivos/ca-2007-02.csv, 1693781398311389>,
 <Blob: hadoop-dados-brutos, anp-combustiveis-automotivos/ca-2008-01.csv, 1693781397510159>,
 <Blob: hadoop-dados-brutos, anp-combustiveis-automotivos/ca-2008-02.csv, 1693781398

Usamos a API para fazer o download dos arquivos com extensão CSV

In [5]:
import re
import os

def download_blob(blob):
    file_path = f'../work/{blob.name}'
        
    # Cria a pasta se não existir
    dir_name = os.path.dirname(file_path)
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
        
    # Download
    blob.download_to_filename(file_path)    

for blob in blobs:
    # Se arquivo CSV
    if re.search(r'.csv$', blob.name):
        print(f'Downloading {blob.name}')
        download_blob(blob)
